In [ ]:
!pip install ta
!pip install load_dotenv

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.10.2-py3-none-any.whl size=29089 sha256=e48d57ac942dbfcfa517789696c8765c42fa361611747c61135c94edc9d5e047
  Stored in directory: /root/.cache/pip/wheels/47/51/06/380dc516ea78621870b93ff65527c251afdfdc5fa9d7f4d248
Successfully built ta


In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import time
from IPython.display import clear_output
import datetime
import pytz
import ta
from statsmodels.tsa.arima.model import ARIMA
import pickle
import csv
import os
from dotenv import load_dotenv
import math

In [ ]:
# Cargar las variables de entorno desde el archivo .env
load_dotenv('/content/drive/MyDrive/ProfeAlejo/Challenge_1/config/.env')
API_KEY = os.getenv('API_KEY')  # API Key de Polygon.io

In [ ]:
def get_df_bitcoin(coin):

  symbol = coin
  period = 2000 # número de dias históricos a obtener
  interval = 5 # intervalo de minutos
  exchange = "USD"

  url = f'https://min-api.cryptocompare.com/data/v2/histominute?fsym={symbol}&tsym={exchange}&aggregate={interval}&limit={period}&api_key={API_KEY}'

  # df = requests.get(url).json()['Data']

  response = requests.get(url)

  data = response.json()


  df = pd.DataFrame(data["Data"]["Data"])

  # Convierte el valor de la columna `time` a un objeto `datetime`.
  df["time"] = pd.to_datetime(df["time"], unit="s")

  # Formatee el valor de la columna `time` en el formato `"%Y-%m-%d %H:%M"`.
  df["time"] = df["time"].dt.strftime("%H:%M")

  df['close'] = df['close'].astype(int)

  df = df[["time", "high",'low','open','volumefrom','volumeto', 'close']]

  # Guardar los datos en un dataframe
  df_bitcoin = pd.DataFrame(df)

  # Calcular indicadores
  df_bitcoin["rsi"] = (ta.momentum.RSIIndicator(df_bitcoin["close"]).rsi()).round(0)
  df_bitcoin["macd"] = (ta.trend.MACD(df_bitcoin["close"]).macd()).round(0)
  df_bitcoin["macd_signal"] = (ta.trend.MACD(df_bitcoin["close"]).macd_signal()).round(0)

  return df_bitcoin

In [ ]:
def get_df_bitcoin_limpio(coin):

    df_data=get_df_bitcoin(coin)

    # 2 - Crear df con columnas: DataTime - Close - Volume
    df_bitcoin_limpio= df_data[['close','volumeto','rsi','macd','macd_signal']]

    # 3 - Eliminar filas con volume=0
    df = df_bitcoin_limpio[df_bitcoin_limpio['volumeto'] != 0]

    # 4 - Encontrar los índices duplicados
    duplicados = df.index.duplicated()

    # Eliminar las filas duplicadas
    df = df[~duplicados]

    # 5 - Elimina filas con Close=0
    df_bitcoin_limpio = df[df['close'] != 0]

    return df_bitcoin_limpio

In [ ]:
def between_quartiles(coin):
    df=get_df_bitcoin_limpio(coin)
    cuartiles = [0, 0.25, 0.5, 0.75, 1]

    try:
        valores = df['close'].quantile(cuartiles)
    except ValueError:
        return None

    if math.isnan(valores[0.25]) or math.isnan(valores[0.75]):
        return None

    df_bitcoin_cuartiles = df.query('close > ' + str(int(valores[0.25])) + ' & close < ' + str(int(valores[0.75])))

    return df_bitcoin_cuartiles

In [ ]:
def extraer_tendencias(btc):
    global precio_actual, tendencia

    # Hacer la petición GET a la página web
    page = requests.get(f'https://www.coincarp.com/currencies/{btc}')

    # Crear el objeto BeautifulSoup a partir del HTML de la página web
    soup = BeautifulSoup(page.content, 'html.parser')

    # Encontrar el precio actual del Bitcoin
    precio = soup.find('span', {'id': 'coin-lastticker'}).text.strip().replace('$', '').replace(',', '')
    precio_actual = float(precio)

    # Análisis de tendencia a partir del icono up o down
    span_element = soup.find('div', {'class': ['cryptocurrencies-price d-flex align-items-center']}).find('button')
    i_element = span_element.find('i')

    if 'icon iconfont icon-solid-arrow-up' in i_element['class']:
      tendencia = 'alta'
    else:
      tendencia = 'baja'

    # Encontrar la variación del precio 24hs
    porcentaje = span_element.text.strip().replace('$', '').replace(',', '')

    return [precio_actual, tendencia, porcentaje]

In [ ]:
def tomar_decisiones(coin,btc):
    global symbol, period, interval, df_bitcoin, precio_actual, tendencia, media,rsi,macd,macdsignal,ma50,ma200

    df=between_quartiles(coin)
    if df is None:
        decision = "Sin datos"
        explicacion = "Explicación: No hay suficientes datos para tomar una decisión clara y asertiva. Los datos RSI, MACD y SiGNAL no cubren lo suficiente. Se recomienda buscar otra moneda o esperar a que haya más datos disponibles."
        return f"La decisión para este caso es: {decision}, ya que su {explicacion}"

    df = df[ ['close','volumeto','rsi','macd','macd_signal'] ] # ordenamos columnas

    # Último RSI
    rsi=df.iat[-1, 2]
    # Últimol MACD
    macd=df.iat[-1, 3]
    # Último MACD-SIGNAL
    macdsignal = df.iat[-1, 4]

    # Calcular la media móvil de 50 días y 200 días
    df["ma50"] = df["close"].rolling(50).mean()
    df["ma200"] = df["close"].rolling(200).mean()

    ma50=df["ma50"].iloc[-1]
    ma200=df["ma200"].iloc[-1]

    #Precio actual
    precio = extraer_tendencias(btc)[0]
    #Tendencia actual
    tendencia=extraer_tendencias(btc)[1]
    #media
    media = df['close'].mean()

    # Aplicar el criterio de decisión

    if precio > media and rsi > 50 and macd > macdsignal and precio > ma50 and precio > ma200:
        decision = "Comprar"
        explicacion ="Explicación: El precio actual está por encima de la media, el RSI es alto, el MACD está por encima de su señal,\n el precio actual está por encima de la MA50 y la MA200, lo que indica una tendencia alcista fuerte y una buena oportunidad para comprar."
    elif precio < media and rsi < 50 and macd < macdsignal and precio < ma50 and precio < ma200:
        decision = "Vender"
        explicacion ="Explicación: El precio actual está por debajo de la media, el RSI es bajo, el MACD está por debajo de su señal,\n el precio actual está por debajo de la MA50 y la MA200, lo que indica una tendencia bajista fuerte y una buena oportunidad para vender."
    else:
      decision = "Mantener"
      if tendencia == "Alta":
          if precio > media and precio > ma50 and precio > ma200:
              explicacion ="Explicación: El precio actual está por encima de la media, la MA50 y la MA200, lo que indica una tendencia alcista. Aunque el RSI y el MACD no son muy altos, se recomienda mantener ya que la tendencia general es positiva y puede haber potencial para ganancias adicionales."

          elif precio < media and precio < ma50 and precio < ma200:
                explicacion ="Explicación: El precio actual está por debajo de la media, la MA50 y la MA200, lo que indica una tendencia bajista. Aunque el RSI y el MACD no son muy bajos, se recomienda mantener ya que la tendencia general es negativa y puede haber potencial para disminución adicional en las pérdidas."

          else:
              explicacion ="Explicación: Aunque la tendencia general es alcista, el precio actual no está lo suficientemente por encima de la media, la MA50 y la MA200 para justificar una compra adicional. Sin embargo, tampoco hay señales fuertes de venta, por lo que se recomienda mantener y seguir observando."

      elif tendencia == "Baja":
          if precio < media and precio < ma50 and precio < ma200:
              explicacion ="Explicación: El precio actual está por debajo de la media, la MA50 y la MA200, lo que indica una tendencia bajista. Aunque el RSI y el MACD no son muy bajos, se recomienda mantener ya que la tendencia general es negativa y puede haber potencial para disminución adicional en las pérdidas."

          elif precio > media and precio > ma50 and precio > ma200:
               explicacion ="Explicación: El precio actual está por encima de la media, la MA50 y la MA200, lo que indica una tendencia alcista. Aunque el RSI y el MACD no son muy altos, se recomienda mantener ya que la tendencia general es positiva y puede haber potencial para disminución adicional en las pérdidas."

      else:
          explicacion ="Explicación: Tendencia bajista pero precio no está lo suficientemente por debajo de media, MA50 y MA200 para justificar una venta adicional. Se recomienda mantener y observar."

    return f"La decisión para este caso es: {decision}, ya que su {explicacion}"

In [ ]:
def precios_medias(coin, btc):

    medias = None

    #Data Frame
    data = get_df_bitcoin_limpio(coin)
    if data.empty:
      return "No hay suficientes datos para calcular las Medias Moviles"

    #Calcular media:
    mean_price = data["close"].mean()

    #Desicion:
    decision=tomar_decisiones(coin,btc)[0]

    # Calcular indicadores técnicos
    data["MA50"] = data["close"].rolling(window=50).mean()
    data["MA200"] = data["close"].rolling(window=200).mean()

    # Crear figura
    fig1, ax1 = plt.subplots(figsize=(12,6))

    # Graficar los precios con las medias móviles
    data["close"].plot(ax=ax1, color="black", label="Precio", linewidth=0.7)
    data["MA50"].plot(ax=ax1, color="blue", label="Media Móvil de 50 días", linewidth=0.5)
    data["MA200"].plot(ax=ax1, color="red", label="Media Móvil de 200 días", linewidth=0.5)
    ax1.axhline(y=mean_price, color="green", linestyle="-",label='Precio Medio', linewidth=0.5)
    ax1.legend(loc="best",fontsize=9)
    ax1.set_xlabel("Fecha")
    ax1.set_ylabel("Precio (USD)")
    ax1.set_title(f"{btc} Precios con Medias Móviles")

    #Texto explicativo de la grafica:
    text_price_medias_01= f"Interpretación: Esta gráfica muestra el precio histórico de {btc} en color gris, junto con las medias móviles"
    text_price_medias_02="de 50 y 200 días en azul y rojo, respectivamente. "
    text_price_medias_03= "La media móvil de 50 días se utiliza comúnmente como un indicador de tendencia a corto plazo, mientras que la media"
    text_price_medias_04="móvil de 200 días se utiliza como un indicador de tendencia a largo plazo."

    # Mostrar decision en grafico
    ax1.text(0.5, 0.95, f"Decisión: {decision}", transform=ax1.transAxes, fontsize=20, color='red', verticalalignment='top',horizontalalignment='center', bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))

    ax1.text(0, -0.2,text_price_medias_01, transform=ax1.transAxes, fontsize=10, ha="left")
    ax1.text(0, -0.25,text_price_medias_02 , transform=ax1.transAxes, fontsize=10, ha="left")
    ax1.text(0, -0.3,text_price_medias_03, transform=ax1.transAxes, fontsize=10, ha="left")
    ax1.text(0, -0.35,text_price_medias_04 , transform=ax1.transAxes, fontsize=10, ha="left")

    # Guardar la figura
    medias = plt.show()

    return medias

In [ ]:
def rsi_tendencias(coin,btc):
    #Data Frame
    df = get_df_bitcoin_limpio(coin)
    df = df[ ['close','volumeto','rsi','macd','macd_signal'] ] # ordenamos columnas
    if df.empty:
      return "No hay suficientes datos para calcular el RSI"

    # Calcular el RSI
    rsi = df.iat[-1, 2]

    # Calcular indicadores técnicos
    df["RSI"] = df['rsi']

    # Crear figuras
    fig2, ax2 = plt.subplots(figsize=(10,4))

    # Graficar el RSI
    df["RSI"].plot(ax=ax2, color="purple", label='Índice de Fuerza Relativa',linewidth=0.7)
    ax2.axhline(y=70, color="green", linestyle="--",label='Nivel de Sobrecompra: 70',linewidth=0.9)
    ax2.axhline(y=30, color="red", linestyle="--",label='Nivel de Sobreventa: 30',linewidth=0.9)
    ax2.legend(loc="best",fontsize=9)
    ax2.set_xlabel("Fecha")
    ax2.set_ylabel("RSI")
    ax2.set_title("RSI")

    #Texto explicativo de RSI:
    text_RSI_01= f"Interpretación: Esta gráfica muestra el Indice de Fuerza Relativa (RSI) de {btc} en color morado. Los valores del RSI"
    text_RSI_02= "oscilan entre 0 y 100, y los niveles de sobrecompra y sobreventa se definen típicamente en 70 y 30, respectivamente."
    text_RSI_03= f"Cuando el RSI se acerca a los 30, se considera que el precio de {btc} está sobrevendido y podría aumentar en el futuro."
    text_RSI_04= f"Cuando el RSI se acerca a los 70, se considera que el precio de {btc} está sobrecomprado y podría disminuir en el futuro."

    ax2.text(0, -0.20,text_RSI_01, transform=ax2.transAxes, fontsize=10, ha="left")
    ax2.text(0, -0.25,text_RSI_02 , transform=ax2.transAxes, fontsize=10, ha="left")
    ax2.text(0, -0.3,text_RSI_03, transform=ax2.transAxes, fontsize=10, ha="left")
    ax2.text(0, -0.35,text_RSI_04 , transform=ax2.transAxes, fontsize=10, ha="left")

    # Guardar la figura
    rsi = fig2

    # Mostrar la gráfica
    return rsi

In [ ]:
def macd_tendencias(coin,btc):
    #Data Frame
    df = get_df_bitcoin_limpio(coin)
    df = df[ ['close','volumeto','rsi','macd','macd_signal'] ] # ordenamos columnas
    if df.empty:
      return "No hay suficientes datos para calcular el MACD"

    # Calcular el MACD
    macd = df.iat[-1, 3]

    # Calcular el MACD-SIGNAL
    macdsignal = df.iat[-1, 4]

    # Calcular indicadores técnicos
    df["MACD"]= df['macd']
    df["M-Signal"] =df['macd_signal']

    # Crear figuras
    fig3, ax3 = plt.subplots(figsize=(12,6))

    # Graficar el MACD y la MACD-Señal
    df["MACD"].plot(ax=ax3, color="blue", label="MACD: Diferencia entre MA de 12 y 26 días",linewidth=0.7)
    df["M-Signal"].plot(ax=ax3, color="red", label="M-Signal: : MA de 9 días del MACD",linewidth=0.7)
    ax3.axhline(y=0, color="black", linestyle="--",linewidth=0.7)
    ax3.legend(loc="best",fontsize=9)
    ax3.set_xlabel("Fecha")
    ax3.set_ylabel("MACD")
    ax3.set_title("MACD y Señal")

    #Texto explicativo de MACD y la MACD-Señal:
    text_MACD_01= f"El gráfico muestra el indicador MACD y su señal para el precio de {btc}. El MACD es un indicador que se utiliza para"
    text_MACD_02= "identificar cambios en la tendencia y la fuerza de los movimientos de los precios. Se calcula a partir de la diferencia"
    text_MACD_03= "entre dos promedios móviles exponenciales de diferentes periodos. La línea de señal es una media móvil exponencial del MACD."
    text_MACD_04= "Cuando la línea del MACD cruza por encima de la línea de señal, es una señal alcista, y cuando cruza por debajo, es una señal bajista."

    # Agregar cuadro de texto debajo del gráfico
    ax3.text(0, -0.2,text_MACD_01, transform=ax3.transAxes, fontsize=10, ha="left", color='black')
    ax3.text(0, -0.25,text_MACD_02, transform=ax3.transAxes, fontsize=10, ha="left", color='black')
    ax3.text(0, -0.3,text_MACD_03, transform=ax3.transAxes, fontsize=10, ha="left", color='black')
    ax3.text(0, -0.35,text_MACD_04 , transform=ax3.transAxes, fontsize=10, ha="left", color='black')

    # Guardar la figura
    macd = fig3

    # Mostrar la gráfica
    return macd

In [ ]:
def predecir(coin):

  symbol = coin
  period = 2000 # número de dias históricos a obtener
  interval = 1 # intervalo de minutos
  exchange = "USD"

  url = f'https://min-api.cryptocompare.com/data/v2/histohour?fsym={symbol}&tsym={exchange}&aggregate={interval}&limit={period}&api_key={API_KEY}'
  response = requests.get(url)

  data = response.json()
  df = pd.DataFrame(data["Data"]["Data"])
  df["time"] = pd.to_datetime(df["time"], unit="s")
  df = df.set_index("time")
  df_btc_close = df['close']

  # Ajustar un modelo ARIMA
  model_pkl = ARIMA(df_btc_close, order=(5, 5, 5))  # Ejemplo con ARIMA(5,1,0)
  results = model_pkl.fit()

  # Ruta para guardar el modelo y los resultados
  ruta_modelo = '/content/drive/MyDrive/ProfeAlejo/Challenge_1/modelo_arima.pkl'

  # Guardar el modelo y los resultados en un archivo
  with open(ruta_modelo, 'wb') as archivo_modelo:
      pickle.dump({'model': model_pkl, 'results': results}, archivo_modelo)

  """#Extraccion del Modelo de la raiz:"""

  # Ruta al archivo donde guardaste el modelo y los resultados
  save_modelo = '/content/drive/MyDrive/ProfeAlejo/Challenge_1/modelo_arima.pkl'

  # Cargar el modelo y los resultados
  with open(save_modelo, 'rb') as archivo_modelo:
      modelo_y_resultados = pickle.load(archivo_modelo)

  # Recuperar el modelo y los resultados del diccionario
  model_cargado = modelo_y_resultados['model']
  results = modelo_y_resultados['results']

  """# Realizar predicciones"""

  #Ultimo valor del BTC
  df_last_close=df['close'].tail(3)
  df_last_close=pd.DataFrame(df_last_close)

  # Realizar predicciones
  forecast_steps = 25  # Por ejemplo, pronosticar 10 pasos en el futuro

  # Realiza las predicciones
  forecast = results.get_forecast(steps=forecast_steps)

  # Extraer las predicciones y los intervalos de confianza
  forecasted_values = forecast.predicted_mean
  confidence_intervals = forecast.conf_int()

  df_predicciones=pd.DataFrame(forecasted_values)

  return f"Ultimos datos de Close {df_last_close}\n Predicciones {df_predicciones}"

In [ ]:
def grafico_predecir(coin):

  fig4 = None

  symbol = coin
  period = 2000 # número de dias históricos a obtener
  interval = 1 # intervalo de minutos
  exchange = "USD"

  url = f'https://min-api.cryptocompare.com/data/v2/histohour?fsym={symbol}&tsym={exchange}&aggregate={interval}&limit={period}&api_key={API_KEY}'
  response = requests.get(url)

  data = response.json()
  if 'Response' in data and data['Response'] == 'Error':
    return "Validando datos en Cryptocompare..."

  df = pd.DataFrame(data["Data"]["Data"])
  df["time"] = pd.to_datetime(df["time"], unit="s")
  df = df.set_index("time")
  df_btc_close = df['close']

  # Ruta al archivo donde guardaste el modelo y los resultados
  save_modelo = '/content/drive/MyDrive/ProfeAlejo/Challenge_1/modelo_arima.pkl'

  # Cargar el modelo y los resultados
  with open(save_modelo, 'rb') as archivo_modelo:
      modelo_y_resultados = pickle.load(archivo_modelo)

  # Recuperar el modelo y los resultados del diccionario
  model_cargado = modelo_y_resultados['model']
  results = modelo_y_resultados['results']

  # Graficar las predicciones y los datos originales
  df_btc_close = df['close']

  # Realizar predicciones
  forecast_steps = 5  # Por ejemplo, pronosticar 10 pasos en el futuro
  forecast = results.get_forecast(steps=forecast_steps)

  # Extraer las predicciones y los intervalos de confianza
  forecasted_values = forecast.predicted_mean
  confidence_intervals = forecast.conf_int()

  # Crear forecast_dates y ajustar las etiquetas
  last_date = df_btc_close.index[-1]
  forecast_dates = pd.date_range(start=last_date, periods=forecast_steps, freq='H')  # Nota que eliminamos el paso adicional
  date_labels = [date.strftime('%Y-%m-%d %H:%M') for date in forecast_dates]  # Incluye la hora y minutos

  # Graficar las predicciones y los datos originales
  ultimas_horas=100
  fig4 = plt.figure(figsize=(14, 7))
  ax4 = fig4.add_subplot(111)
  ax4.plot(df_btc_close.tail(ultimas_horas), label='Datos Originales', color='blue')
  ax4.plot(forecast_dates, forecasted_values, label='Predicciones', color='red')
  ax4.fill_between(confidence_intervals.index, confidence_intervals.iloc[:, 0], confidence_intervals.iloc[:, 1], color='red', alpha=0.3, label='Intervalo de Confianza')

  ax4.legend()
  ax4.set_title(f'Predicciones de {coin}-USD con ARIMA por hora')

  # Devolver el objeto Figure
  return data

In [ ]:
def mayus(coin):
    coin = coin.replace(" ", "")
    coin = coin.upper()
    return coin

In [ ]:
web = 'https://www.coincarp.com/'

def get_crypto_data():
    all_names = []
    all_links = []

    page = 1
    while True:
        # print(f'Processing page {page}')

        response = requests.get(f"{web}pn_{page}.html")
        soup = BeautifulSoup(response.text, "html.parser")

        for span in soup.find_all("span", class_="symbo"):

          text = ""

          for child in span.children:
            if child.name != "i":
              text += child.strip()

          if text:
            all_names.append(text)

        links = soup.find_all("td", class_="td2 sticky")

        for link in links:
          a_tag = link.find("a")
          href = a_tag["href"]
          if href not in all_links:
            all_links.append(href)

        if page > 19:
            break

        page += 1

    cleaned_links = []
    for link in all_links:
        link = link.replace("/currencies/", "")
        link = link.rstrip("/")
        cleaned_links.append(link)

    name_to_link = {}
    for name, link in zip(all_names, cleaned_links):
        name_to_link[name] = link

    return name_to_link

In [ ]:
def get_crypto_info(coin):

  name_to_link = get_crypto_data()

  # Guardar el diccionario en CSV
  with open('/content/drive/MyDrive/ProfeAlejo/Challenge_1/crypto_data.csv', 'w') as f:
    writer = csv.writer(f)
    for name, link in name_to_link.items():
      writer.writerow([name, link])

  # Leer CSV
  with open('/content/drive/MyDrive/ProfeAlejo/Challenge_1/crypto_data.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
      if coin in row:
        return row[1]
  # f.close()
  return None

In [ ]:
def automatizar(coin,btc):
  get_df_bitcoin(coin)
  get_df_bitcoin_limpio(coin)
  between_quartiles(coin)
  extraer_tendencias(btc)
  tomar = tomar_decisiones(coin,btc)
  grafico1 = precios_medias(coin, btc)
  grafico2 = rsi_tendencias(coin, btc)
  grafico3 = macd_tendencias(coin,btc)
  predicion = predecir(coin)
  grafico_predicion = grafico_predecir(coin)

  return tomar, grafico1, grafico2, grafico3, predicion, grafico_predicion

In [ ]:
def extraer():
    web = "https://coinmarketcap.com"

    page = requests.get(web)

    soup = BeautifulSoup(page.content, "html.parser")

    # Extraer los datos de los elementos HTML
    imagenes = soup.find_all("img", class_="coin-logo")
    enlaces = soup.find_all("div", class_="sc-aef7b723-0 LCOyB")
    nombres = soup.find_all("p", class_="sc-4984dd93-0 kKpPOn")
    monedas = soup.find_all("p", class_="sc-4984dd93-0 iqdbQL coin-item-symbol")

    # Extraer los datos del dataframe de Python en un diccionario
    datos = {
        "imagen": [imagen["src"] for imagen in imagenes],
        "web_completa": [f'{web}{enlace.find("a")["href"]}' for enlace in enlaces],
        "moneda_nombre": [
            moneda.text + " - " + nombre.text
            for moneda, nombre in zip(monedas, nombres)
        ],
    }

    return datos

In [ ]:
def generar_html(datos):
    html = ""

    for i in range(len(datos["imagen"])):
        html += f"""
        <span href="{datos['web_completa'][i]}">
          <img src="{datos['imagen'][i]}" />
          <a>{datos['moneda_nombre'][i]}</a>
        </span>
    """

    return html

In [ ]:
# coin = mayus("btc")
# btc = get_crypto_info(coin)

# print(automatizar(coin, btc))

In [ ]:
# !pip freeze > requirements.txt